In [1]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pathlib
# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3468, done.
remote: Counting objects: 100% (3468/3468), done.
remote: Compressing objects: 100% (2789/2789), done.
remote: Total 3468 (delta 1042), reused 1523 (delta 630), pack-reused 0
Receiving objects: 100% (3468/3468), 34.30 MiB | 33.65 MiB/s, done.
Resolving deltas: 100% (1042/1042), done.


In [3]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1686356 sha256=9fb6fd3da327331ce58b0c7c468049d179565872b6ac7be6dd2bfd4f5a5a5853
  Stored in directory: /tmp/pip-ephem-wheel-cache-fv8g97qc/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=8ddd602ec89153b31440b42b641db13e6e13246609758ea52d964a48bcffc834
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=4954d9d9c71d751ed5293c737d45242419128927576de5e410d8aad807c0d764
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.whl size=44010 sha256=419f5d5d2292b97ee91c06888080224c6626e12877fbe48190

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
gym 0.17.3 requires cloudpickle<1.7.0,>=1.2.0, but you have cloudpickle 2.0.0 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [4]:
import matplotlib
import matplotlib.pyplot as plt
import time
import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

In [5]:
NUM_CLASSES = 8

category_index = {1: {'id': 1, 'name': 'bone'},
                  2: {'id': 2, 'name': 'abdomen'},
                  3: {'id': 3, 'name': 'mediastinum'},
                  4: {'id': 4, 'name': 'liver'},
                  5: {'id': 5, 'name': 'lung'},
                  6: {'id': 6, 'name': 'kidney'},
                  7: {'id': 7, 'name': 'soft_tissue'},
                  8: {'id': 8, 'name': 'pelvis'}}

def _parse_function(example_proto):
    feature_description = {
        'image/height': tf.io.FixedLenFeature((), tf.int64, default_value=1),
        'image/width': tf.io.FixedLenFeature((), tf.int64, default_value=1),
        'image/win_min': tf.io.FixedLenFeature((), tf.float32, default_value=-1024.),
        'image/win_max': tf.io.FixedLenFeature((), tf.float32, default_value=3071.), 
        'image/filename': tf.io.FixedLenFeature((), tf.string, default_value=''),
        'image/source_id': tf.io.FixedLenFeature((), tf.string, default_value=''),    
        'image/encoded': tf.io.FixedLenFeature((), tf.string, default_value=''),
        'image/format': tf.io.FixedLenFeature((), tf.string, default_value='jpeg'),
        # Object boxes and classes.
        'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
        'image/object/class/text': tf.io.VarLenFeature(tf.string),
        'image/object/class/label': tf.io.VarLenFeature(tf.int64),
    }  
    return tf.io.parse_single_example(example_proto, feature_description)

def GetImageLabelBoxTensors(image_features, batch_size):
    train_image_tensors = []
    gt_box_tensors = []
    gt_classes_one_hot_tensors = [] 
    num_classes = NUM_CLASSES
    label_id_offset = 1
    for i in range(batch_size):
#       print('i = {}'.format(i))
        image = tf.io.decode_png(image_features['image/encoded'][i], channels=3)
        image = tf.cast(image, tf.float32)
        image = tf.expand_dims(image, axis=0)
        xmins = tf.sparse.to_dense(image_features['image/object/bbox/xmin'])[i:i+1]
        xmaxs = tf.sparse.to_dense(image_features['image/object/bbox/xmax'])[i:i+1]
        ymins = tf.sparse.to_dense(image_features['image/object/bbox/ymin'])[i:i+1]
        ymaxs = tf.sparse.to_dense(image_features['image/object/bbox/ymax'])[i:i+1]
        bbox = tf.concat([ymins, xmins, ymaxs, xmaxs], 0)        
        bbox = tf.transpose(bbox)
        labels_np = tf.sparse.to_dense(image_features['image/object/class/label'])[i].numpy()
        zero_args = np.argwhere(labels_np == 0)
        if zero_args.size > 0:
            num_bbox = zero_args[0][0]
            labels_np = labels_np[:num_bbox]
            bbox = bbox[:num_bbox] 
#            print('labels = {}'.format(labels_np))
        zero_indexed_groundtruth_classes = tf.convert_to_tensor(labels_np - label_id_offset)
        gt_classes_one_hot_tensors.append(tf.one_hot(zero_indexed_groundtruth_classes, num_classes))
        train_image_tensors.append(image)
        gt_box_tensors.append(bbox)    
    return train_image_tensors, gt_box_tensors, gt_classes_one_hot_tensors


# Set up forward + backward pass for a single train step.
def get_model_train_step_function(model, optimizer, vars_to_fine_tune):
  """Get a tf.function for training step."""

  # Use tf.function for a bit of speed.
  # Comment out the tf.function decorator if you want the inside of the
  # function to run eagerly.
  @tf.function
  def train_step_fn(image_tensors,
                    groundtruth_boxes_list,
                    groundtruth_classes_list):
    """A single training iteration.

    Args:
      image_tensors: A list of [1, height, width, 3] Tensor of type tf.float32.
        Note that the height and width can vary across images, as they are
        reshaped within this function to be 640x640.
      groundtruth_boxes_list: A list of Tensors of shape [N_i, 4] with type
        tf.float32 representing groundtruth boxes for each image in the batch.
      groundtruth_classes_list: A list of Tensors of shape [N_i, num_classes]
        with type tf.float32 representing groundtruth boxes for each image in
        the batch.

    Returns:
      A scalar tensor representing the total loss for the input batch.
    """
    shapes = tf.constant(batch_size * [[640, 640, 3]], dtype=tf.int32)
    model.provide_groundtruth(
        groundtruth_boxes_list=groundtruth_boxes_list,
        groundtruth_classes_list=groundtruth_classes_list)
    with tf.GradientTape() as tape:
      preprocessed_images = tf.concat(
          [detection_model.preprocess(image_tensor)[0]
           for image_tensor in image_tensors], axis=0)
      prediction_dict = model.predict(preprocessed_images, shapes)
      losses_dict = model.loss(prediction_dict, shapes)
      total_loss = losses_dict['Loss/localization_loss'] + losses_dict['Loss/classification_loss']
      gradients = tape.gradient(total_loss, vars_to_fine_tune)
      optimizer.apply_gradients(zip(gradients, vars_to_fine_tune))
    return total_loss

  return train_step_fn

def get_latest_ckpt(dir):
    search = dir + '/*.index'
    ckpts = glob.glob(search)
    lastest_ckpt = ''
    max_idx = -1
    for item in ckpts:
        item1 = item[:item.index('.')]
        idx = int(item1[item1.index('-')+1:])
        #print(item1, idx)
        if (idx > max_idx):
            max_idx = idx
            lastest_ckpt = item1
    return lastest_ckpt

def get_loss(filename):
    with open(filename, 'r') as f:
        l = [[float(num) for num in line.split()] for line in f]
    return np.array(l)

def plot_loss(in_path, save_path=None):
    f = open(in_path, 'r')
    l = np.array([[float(num) for num in line.split()] for line in f])
    f.close()
    x = l[:,0].astype(int)
    plt.figure()
    plt.grid('on')
    plt.plot(x, l[:,1], 'r')
    plt.text(x[0], l[:,1][0], 'loss')
    if (save_path):
        plt.savefig(save_path)
    else:
        plt.show()

In [6]:
tfr_path = glob.glob('/content/drive/MyDrive/data1/tfr_deeplesion.record-*')
batch_size = 1
dataset = tf.data.TFRecordDataset(tfr_path)
dataset = dataset.map(_parse_function)
dataset = dataset.batch(batch_size) 

In [7]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!tar -xf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!mv ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint models/research/object_detection/test_data/

--2022-03-22 02:50:13--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.218.128, 2607:f8b0:400c:c14::80
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.218.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244817203 (233M) [application/x-tar]
Saving to: ‘ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz’

ssd_resnet50_v1_fpn 100%[===================>] 233.48M   168MB/s    in 1.4s    

2022-03-22 02:50:15 (168 MB/s) - ‘ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz’ saved [244817203/244817203]



In [8]:
tf.keras.backend.clear_session()

print('Building model and restoring weights for fine-tuning...', flush=True)
num_classes = NUM_CLASSES
pipeline_config = 'models/research/object_detection/configs/tf2/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config'
checkpoint_path = get_latest_ckpt('/content/drive/MyDrive/my_model')

# Load pipeline config and build a detection model.
#
# Since we are working off of a COCO architecture which predicts 90
# class slots by default, we override the `num_classes` field here to be just
# one (for our new rubber ducky class).
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
model_config.ssd.num_classes = num_classes
model_config.ssd.freeze_batchnorm = True
detection_model = model_builder.build(model_config=model_config, is_training=True)

ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
if (len(checkpoint_path) > 0):
    ckpt.restore(checkpoint_path).expect_partial()

# Run model through a dummy image so that variables are created
image, shapes = detection_model.preprocess(tf.zeros([1, 640, 640, 3]))
prediction_dict = detection_model.predict(image, shapes)
_ = detection_model.postprocess(prediction_dict, shapes)
print('Weights restored!')

Building model and restoring weights for fine-tuning...
Weights restored!


In [11]:
#train
tf.keras.backend.set_learning_phase(True)

batch_size = 1
learning_rate = 0.0001
num_epochs = 100
N = len(list(dataset))

to_fine_tune = []
for var in detection_model.trainable_variables:
    to_fine_tune.append(var)

optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
train_step_fn = get_model_train_step_function(detection_model, optimizer, to_fine_tune)

print('Start fine-tuning! There are {} batches in dataset'.format(N), flush=True)

total_loss = 100.0
epoch = 0
if os.path.exists('/content/drive/MyDrive/my_model/loss.txt'):
    l = get_loss('/content/drive/MyDrive/my_model/loss.txt') 
    if l.size > 0:
        epoch = l[-1][0]
        total_loss = l[-1][1]
    
dataset = dataset.shuffle(1000)
while epoch < num_epochs and total_loss > 0.2:
    total_loss = 0
    tic = time.perf_counter()
    for image_features in dataset:
        image_tensors, gt_boxes_list, gt_classes_list = GetImageLabelBoxTensors(image_features, batch_size)
        loss = train_step_fn(image_tensors, gt_boxes_list, gt_classes_list)
        total_loss += (loss.numpy())
    total_loss /= N
    toc = (time.perf_counter() - tic)/60.
    print('epoch {} of {}, average loss = {:0.4f}, time_cost = {:0.4f} min'.format(epoch, num_epochs, total_loss, toc), flush = True)

    with open('/content/drive/MyDrive/my_model/trainlog.txt', 'a+') as flog:
        flog.write('epoch {} of {}, average loss = {:0.4f}, time_cost = {:0.4f} min\n'.format(epoch, num_epochs, total_loss, toc))
    with open('/content/drive/MyDrive/my_model/loss.txt', 'a+') as fp:
        fp.write('{} {:0.4f}\n'.format(epoch, total_loss))

    if ((epoch + 1) % 5 == 0):
        _ = ckpt.save('/content/drive/MyDrive/my_model/ckpt1')
        print('Model Saved!')
        
    epoch += 1
print('Done fine-tuning!')

with open('/content/drive/MyDrive/my_model/success.txt', 'w') as fp1:
    fp1.write('good!\n')
_ = ckpt.save('/content/drive/MyDrive/my_model/ckpt1')

plot_loss('/content/drive/MyDrive/my_model/loss.txt', '/content/drive/MyDrive/my_model/loss.png')

/usr/local/lib/python3.7/dist-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


Start fine-tuning! There are 7023 batches in dataset
epoch 0 of 100, average loss = 0.8200, time_cost = 15.4240 min


KeyboardInterrupt: ignored